<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/Copy_of_Semi_blind_MLP_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# Generating datasets and preprocessing

In [2]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j = complex(0, 1) #Imaginary part
def fading_channel (Nt, Nr):
  h_real = np.random.normal(0, 0.5, size = (Nr, Nt))
  h_img = np.random.normal(0, 0.5, size = (Nr, Nt))
  H_Rayleigh = np.array(h_real + h_img*j, dtype = complex)
  
  return H_Rayleigh

In [3]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4, 4))
Constellation_set[0] = np.array([1, 0, 0, 0])
Constellation_set[1] = np.array([0, 1, 0, 0])
Constellation_set[2] = np.array([0, 0, 1, 0])
Constellation_set[3] = np.array([0, 0, 0, 1])

In [4]:
H_Rayleigh = fading_channel (4,2)

In [5]:
'''Complex noise'''
def noise_complex (N0, Nr):
  n_real = np.random.normal(0, N0, size = Nr)
  n_img=np.random.normal(0, N0, size = Nr)
  noise_complex=np.array(n_real + n_img*j, dtype = complex)
  
  return noise_complex

In [6]:
'''Creating pilot signals'''
def training_pilots_at_receiver (Label):
  i = 0
  
  y_receiver = np.zeros(shape = (4, 2), dtype = "complex_")
  for i in range(Label):
    y_receiver[i] = np.dot(H_Rayleigh, Constellation_set[i], out = None) + noise_complex (0.125, 2)
    i += 1

  y_receiver_real = y_receiver.real
  y_receiver_imag = y_receiver.imag
  y_receiver = np.concatenate((y_receiver_real, y_receiver_imag), axis = 1)
  y_receiver_complex = pd.DataFrame(y_receiver, columns=['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
  y_receiver_complex['Label'] = np.arange(0, 4) 

  return y_receiver_complex

In [7]:
training_pilots_at_receiver = training_pilots_at_receiver (4)

In [8]:
'''Creating test signals'''
def test_ (packet_length):
  #Define an empty dataframe
  y_receive_test = pd.DataFrame()

  for i in range(packet_length):
    #Generating received signals
    y_one_receive_signal = np.zeros(shape = (1, 2), dtype = "complex_")
    
    #Choosing a random constellation point
    rand = random.randrange(0, 4, 1)
    
    #Y = HX+N 
    y_one_receive_signal = np.dot(H_Rayleigh, Constellation_set[rand], out = None) + noise_complex (0.125, 2)
    
    #Transforming complex signal to a two dimension integer
    y_one_receive_signal_real = y_one_receive_signal.real
    y_one_receive_signal_imag = y_one_receive_signal.imag
    y_receive = np.concatenate((y_one_receive_signal_real,y_one_receive_signal_imag), axis = None)
    
    #Filling dataframe rows
    y_receive_without_label = pd.DataFrame(y_receive.reshape((1, 4)), columns = ['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag'])
    
    #Assigning related label to received pilots
    if rand == 0 :
      y_receive_without_label['Label']= 0
    elif rand == 1 :
      y_receive_without_label['Label']= 1
    elif rand == 2 :
      y_receive_without_label['Label']= 2
    elif rand == 3 :
      y_receive_without_label['Label']= 3
    
    #Join all test signals to data set
    y_receive_test = pd.concat([y_receive_test, y_receive_without_label])

  return y_receive_test

In [9]:
def test_label (KNN_or_NN): #Depend on model
  test = test_ (200)

  #Test data for KNN
  if KNN_or_NN == 1 :
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    return test_features, test_label

  #Test data for NN
  elif KNN_or_NN == 0:
    test_features = test[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']]
    test_label = test['Label']
    test_label_categorical = to_categorical(test_label, num_classes = 4)
    return test_features, test_label_categorical

In [10]:
test_label (1)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0    0.477289  -0.310802   0.051603  -0.905267
 0   -0.568458  -0.514007   0.472715   0.654406
 0    0.597114  -0.123244   0.708140   0.230836
 0    0.622697   0.424735   0.091835   0.005969
 0    0.570806   0.499970   0.203818   0.045949
 ..        ...        ...        ...        ...
 0    0.473130  -0.277434   0.579772   0.103731
 0    0.539695   0.403564   0.395414  -0.147677
 0    0.416850  -0.016903   0.864814  -0.034508
 0    0.413672   0.669421   0.211405  -0.359897
 0    0.471310  -0.142767   0.511670   0.293930
 
 [200 rows x 4 columns], 0    1
 0    0
 0    3
 0    2
 0    2
     ..
 0    3
 0    2
 0    3
 0    2
 0    3
 Name: Label, Length: 200, dtype: int64)

In [11]:
test_label (0)

(    Ant1_real  Ant2_real  Ant1_imag  Ant2_imag
 0   -0.529378  -0.415758   0.487021   0.626086
 0    0.645665  -0.088325   0.130522  -0.974428
 0    0.423440  -0.147977   0.652170   0.099311
 0    0.476588   0.568266   0.202508  -0.202732
 0    0.365248   0.079458   0.796021   0.198768
 ..        ...        ...        ...        ...
 0    0.580577   0.513170   0.347229  -0.000606
 0    0.467000  -0.421636  -0.170278  -0.693808
 0   -0.539157  -0.317080   0.644253   0.674635
 0   -0.520211  -0.295581   0.529492   0.550386
 0    0.581981  -0.301690  -0.285489  -0.835393
 
 [200 rows x 4 columns], array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        

In [12]:
x_test_svm, y_test_svm = test_label (1)
x_test_nn, y_test_nn = test_label (0)


# Generating virtual pilots (data augmentation)

In [13]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  noise_real=np.random.normal(0, N0, size=Nr)
  
  return noise_real

In [14]:
def data_augmentation (sample_size, noise_variance):
  #Generating virtual pilots based on actual pilots
  augmented_data = pd.DataFrame()

  for i in range(sample_size):
    frame_augmented = training_pilots_at_receiver[['Ant1_real', 'Ant2_real', 'Ant1_imag', 'Ant2_imag']] + noise_real(noise_variance, 4)
    frame_augmented['Label'] = np.arange(0, 4)
    augmented_data = augmented_data.append(frame_augmented)
    i += 1

  return augmented_data

In [15]:
data_augmentation = data_augmentation(300, 0.06)

In [16]:
print(data_augmentation.describe())
print(training_pilots_at_receiver.describe())

         Ant1_real    Ant2_real    Ant1_imag    Ant2_imag      Label
count  1200.000000  1200.000000  1200.000000  1200.000000  1200.0000
mean      0.338501     0.027417     0.273934     0.030190     1.5000
std       0.398863     0.433486     0.310010     0.499291     1.1185
min      -0.502298    -0.591992    -0.338928    -0.757118     0.0000
25%       0.135909    -0.329983     0.044944    -0.392173     0.7500
50%       0.498152    -0.101113     0.354615    -0.034998     1.5000
75%       0.631097     0.309645     0.539951     0.381153     2.2500
max       0.820100     0.898623     0.752654     0.923022     3.0000
       Ant1_real  Ant2_real  Ant1_imag  Ant2_imag     Label
count   4.000000   4.000000   4.000000   4.000000  4.000000
mean    0.342993   0.029634   0.268600   0.026251  1.500000
std     0.455399   0.495166   0.351766   0.571931  1.290994
min    -0.320795  -0.388098  -0.202019  -0.586703  0.000000
25%     0.238837  -0.298676   0.105114  -0.305829  0.750000
50%     0.502731  -

# Split dataset to train and test

In [17]:
'''Spliting features and labels'''
#xy=pd.concat([data_augmentation, training_pilots_at_receiver])       #Enable when you want vp for SVM & KNN
xy = training_pilots_at_receiver                      #Enable when you dont want vp for SVM & KNN
xy_NN = pd.concat([data_augmentation, training_pilots_at_receiver])     #Enable when you want vp for NN
#xy_NN = training_pilots_at_receiver                   #Enable when you dont want vp for NN

In [18]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.320795,-0.388098,0.582608,0.757155,0
1,0.580081,-0.268869,-0.202019,-0.586703,1
2,0.687304,0.715980,0.207492,-0.212204,2
3,0.425381,0.059521,0.486319,0.146756,3


#Preprocessing for KNN & SVM

In [19]:
scaler = StandardScaler()

In [20]:
scaler.fit(xy.drop('Label',axis=1))

In [21]:
scaled_features = scaler.transform(xy.drop('Label',axis=1))

In [22]:
xy = pd.DataFrame(scaled_features,columns=xy.columns[:-1])
xy['Label']=np.arange(0,4)
xy

#Preprocessing for MLP

In [23]:
xy_NN

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.334541,-0.359698,0.555407,0.808901,0
1,0.566335,-0.240469,-0.229220,-0.534957,1
2,0.673558,0.744380,0.180290,-0.160458,2
3,0.411635,0.087922,0.459118,0.198503,3
0,-0.265096,-0.453052,0.617969,0.721873,0
...,...,...,...,...,...
3,0.531712,0.074944,0.479903,0.143022,3
0,-0.320795,-0.388098,0.582608,0.757155,0
1,0.580081,-0.268869,-0.202019,-0.586703,1
2,0.687304,0.715980,0.207492,-0.212204,2


In [24]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.3, random_state=101)

In [25]:
'''Spliting features and labels for SVM & KNN'''
x=xy[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y=xy['Label']

In [26]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.320795,-0.388098,0.582608,0.757155,0
1,0.580081,-0.268869,-0.202019,-0.586703,1
2,0.687304,0.715980,0.207492,-0.212204,2
3,0.425381,0.059521,0.486319,0.146756,3


In [27]:
'''Spliting data to train and validation'''
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.000001, random_state=101)
X_train=x
y_train=y

In [28]:
X_train.shape

(4, 4)

In [29]:
y_train.shape

(4,)

In [30]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95 and logs.get('val_loss')<0.1):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()

# Building neural network and training

In [31]:
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(32, activation=keras.layers.LeakyReLU(alpha=0.01)))
model.add(Dense(64, activation=keras.layers.LeakyReLU(alpha=0.01)))
#model.add(Dropout(rate=0.1))
#model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(32, activation=keras.layers.LeakyReLU(alpha=0.01)))
model.add(Dense(4, activation='softmax'))


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
NN=model.fit(X_train_NN, y_train_NN, batch_size=32, epochs=10, validation_data=(X_test_NN,y_test_NN),callbacks=[callbacks])

Epoch 1/10
27/27 [==============================] - 4s 47ms/step - loss: 1.1685 - accuracy: 0.8539 - val_loss: 0.9322 - val_accuracy: 0.9365
Epoch 2/10
27/27 [==============================] - 0s 12ms/step - loss: 0.6905 - accuracy: 0.9608 - val_loss: 0.4322 - val_accuracy: 1.0000
Epoch 3/10
27/27 [==============================] - 0s 11ms/step - loss: 0.2694 - accuracy: 0.9988 - val_loss: 0.1109 - val_accuracy: 1.0000
Epoch 4/10
17/27 [=================>............] - ETA: 0s - loss: 0.0820 - accuracy: 1.0000
Reached 99% accuracy so cancelling training!
27/27 [==============================] - 0s 10ms/step - loss: 0.0692 - accuracy: 1.0000 - val_loss: 0.0242 - val_accuracy: 1.0000


In [33]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 4,484
Trainable params: 4,484
Non-trainable params: 0
_________________________________________________________________


# Training KNN and SVM model

In [34]:
rbf = svm.SVC(kernel = 'rbf', gamma = 0.5, C = 0.1).fit(X_train, y_train)
poly = svm.SVC(kernel = 'poly', degree = 3, C = 1).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

# Testing models on test dataset 

In [35]:
poly_pred = poly.predict(x_test_svm)
rbf_pred = rbf.predict(x_test_svm)
knn_pred = knn.predict(x_test_svm)

In [36]:
knn_accuracy = accuracy_score(y_test_svm, knn_pred)
knn_f1 = f1_score(y_test_svm, knn_pred, average ='weighted')
print('Accuracy (KNN): ', "%.2f" % (knn_accuracy*100))
print('F1 (KNN): ', "%.2f" % (knn_f1*100))

Accuracy (KNN):  97.50
F1 (KNN):  97.48


In [37]:
poly_accuracy = accuracy_score(y_test_svm, poly_pred)
poly_f1 = f1_score(y_test_svm, poly_pred, average = 'weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  85.50
F1 (Polynomial Kernel):  85.06


In [38]:
rbf_accuracy = accuracy_score(y_test_svm, rbf_pred)
rbf_f1 = f1_score(y_test_svm, rbf_pred, average = 'weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  97.50
F1 (RBF Kernel):  97.48


In [39]:
'''Test the model'''
NN_Accuracy = model.evaluate(x_test_nn, y_test_nn)

7/7 [==============================] - 0s 5ms/step - loss: 0.0684 - accuracy: 1.0000


#Choosing K value

In [40]:
'''error_rate = []

for i in range(1,4):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(m1)
    error_rate.append(np.mean(pred_i != n1))'''

'error_rate = []\n\nfor i in range(1,4):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(X_train,y_train)\n    pred_i = knn.predict(m1)\n    error_rate.append(np.mean(pred_i != n1))'

In [41]:
'''plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)
plt.title('Error Rate vs. K Values')
plt.xlabel('K')
plt.ylabel('Error Rate')'''

"plt.figure(figsize=(10,6))\nplt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)\nplt.title('Error Rate vs. K Values')\nplt.xlabel('K')\nplt.ylabel('Error Rate')"